In [1]:
import requests

use_real_puzzle_data = False # else use demo data
separator = '\n' if True else ','

github_site_user = "https://raw.githubusercontent.com/stephenhumphrey/"
repository_branch = "advent-of-code/main/2021/12/18/"

names_files = [("magnitudes","demo-magnitudes"),
               ("demo","demo-summations"),
               ("puzzle","puzzle-data")]

data = {source:{"name":source,
                "url":f"{github_site_user}{repository_branch}{file}.txt"} 
        for source,file in names_files}

for name,source in data.items():
    print(name,source["url"])

magnitudes https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/demo-magnitudes.txt
demo https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/demo-summations.txt
puzzle https://raw.githubusercontent.com/stephenhumphrey/advent-of-code/main/2021/12/18/puzzle-data.txt


In [2]:
infinity = 99999999999999999999999999

do_verbose = False
def throwaway_verbose(unused):
    return
verbose = print if do_verbose else throwaway_verbose

In [3]:
def verbose_list(plist_string,max_elements = 5):
    plist = eval(plist_string)
    if len(plist) <= max_elements:
        sub = f"{plist}"
    elif max_elements < 2:
        sub = f"[{plist[0]}, ...]"
    else:
        half_max = max_elements // 2
        sub = "["
        for v in plist[0:half_max]:
            sub += f"{v}, "
        sub += "... "
        for v in plist[-half_max:-1]:
            sub += f"{v}, "
        sub += f"{plist[-1]}]"
        
    print(f"len({plist_string})={len(plist)}, {plist_string}={sub}")

In [4]:
for source_name,source in data.items():
    raw_data = requests.get(source["url"]).text
    data_lines = [line for line in raw_data.split(separator) if len(line.strip()) > 0]
    source["raw_lines"] = data_lines
    
    if False:
        print(source_name)
        for line in data_lines:
            print(line)
    
    if source_name == "puzzle":
        source["operands"] = [data_lines]
        source["expected_values"] = ["unknown"]
    else:
        right = []
        source["operands"] = []
        for line in data_lines:
            line_data = [v.strip() for v in line.split('=') if len(v.strip())>1]
            left = [v.strip() for v in line_data[0].split('+')]
            source["operands"].append(left)
            right.append(line_data[1])
        source["expected_values"] = right

In [5]:
def separate_pair(pair_string):
    assert(pair_string[0]=='[' and pair_string[-1]==']')
    
    depth = 0
    split_at = 0
    for i,c in enumerate(pair_string):
        if c == '[':
            depth += 1
        elif c == ']':
            depth -= 1
        elif c == ',' and depth == 1:
            split_at = i
            break
    return pair_string[1:split_at], pair_string[split_at + 1:-1]

def parse_pair(pair_string):
    
    if pair_string[0] != '[':
        return int(pair_string)
    
    assert(pair_string[0]=='[' and pair_string[-1]==']')

    left,right = separate_pair(pair_string)
    left_parsed = parse_pair(left)
    right_parsed = parse_pair(right)
    
    return (left_parsed,right_parsed)

def pair_magnitude(pair):
    assert(type(pair)==type((9,9)) and len(pair)==2)
    
    left,right = pair
    assert(type(left)==type(9) or type(left)==type((9,9)))
    assert(type(right)==type(9) or type(right)==type((9,9)))
    
    sum_left_right = 0
    if type(left) == type(9):
        sum_left_right += left * 3
    else:
        sum_left_right += pair_magnitude(left) * 3
    
    if type(right) == type(9):
        sum_left_right += right * 2
    else:
        sum_left_right += pair_magnitude(right) * 2

    return sum_left_right

In [6]:
for source_name,source in data.items():
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}) expect:{expected_values[i]}")
        for j,operand in enumerate(operands):
            pair = parse_pair( operand )
            magnitude = pair_magnitude( pair )
            print(f"  operand[{j}] = {operand} pair = {pair} magnitude = {magnitude}")
assert(False)

magnitudes0 operands(1) expect:29
  operand[0] = [9,1] pair = (9, 1) magnitude = 29
magnitudes1 operands(1) expect:21
  operand[0] = [1,9] pair = (1, 9) magnitude = 21
magnitudes2 operands(1) expect:129
  operand[0] = [[9,1],[1,9]] pair = ((9, 1), (1, 9)) magnitude = 129
magnitudes3 operands(1) expect:143
  operand[0] = [[1,2],[[3,4],5]] pair = ((1, 2), ((3, 4), 5)) magnitude = 143
magnitudes4 operands(1) expect:1384
  operand[0] = [[[[0,7],4],[[7,8],[6,0]]],[8,1]] pair = ((((0, 7), 4), ((7, 8), (6, 0))), (8, 1)) magnitude = 1384
magnitudes5 operands(1) expect:445
  operand[0] = [[[[1,1],[2,2]],[3,3]],[4,4]] pair = ((((1, 1), (2, 2)), (3, 3)), (4, 4)) magnitude = 445
magnitudes6 operands(1) expect:791
  operand[0] = [[[[3,0],[5,3]],[4,4]],[5,5]] pair = ((((3, 0), (5, 3)), (4, 4)), (5, 5)) magnitude = 791
magnitudes7 operands(1) expect:1137
  operand[0] = [[[[5,0],[7,4]],[5,5]],[6,6]] pair = ((((5, 0), (7, 4)), (5, 5)), (6, 6)) magnitude = 1137
magnitudes8 operands(1) expect:3488
  oper

AssertionError: 

In [ ]:
for source_name,source in data.items():
    expected_values = source["expected_values"]
    for i,operands in enumerate(source["operands"]):
        print(f"{source_name}{i} operands({len(operands)}):{operands} expect:{expected_values[i]}")
assert(False)